<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow


Step 1: Data Preparation and Preprocessing
DICOM Images

DICOM Handling: Use pydicom to read DICOM files, which contain both the images and embedded metadata. For each DICOM file, extract the image data for preprocessing and the metadata for analysis.
Image Preprocessing: Since DICOM images might not be uniformly sized, you may need to resize them to a standard size (e.g., 1024x1024) to match the TFRecord images, normalize pixel values, and potentially augment the data to improve model robustness.
JPEG and TFRecord Images

JPEG Handling: Directly read and preprocess (resize if necessary, normalize, augment).
TFRecord Handling: Extract and preprocess. TFRecord files are already uniformly sized, which simplifies preprocessing.
Metadata

CSV Metadata: Load the metadata from CSV files. Clean and preprocess this data, which involves handling missing values, normalizing or scaling numerical features, and encoding categorical features.



In [4]:
!pip install isic-cli


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.9 MB/s eta 0:00:00


In [5]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=szpDCj95PH5xL0sjBRruQu8crfkXyc&code_challenge=EEfrSJ2Pzm4yD2XT5ADFX7tEjX2nOTMryjCU47meSr0&code_challenge_method=S256
enter the code shown in your browser: Ww7huyqCiRXfHvnJakpInteV4eIi8a
Success!


Step 2: Query the Dataset
Endpoints: Use the API endpoints to query the dataset. Common operations include listing available images, retrieving image metadata, and downloading images.
Filtering: Utilize query parameters to filter the dataset based on your criteria, such as diagnosis, image type, or other metadata.

In [6]:
!pip install isic-cli


In [28]:
! pip install kaggle

In [31]:
!kaggle competitions download -c titanic

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [7]:
!isic collection list


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None            │
│ 64  │ Challenge 2018: Task 1-2: Test                │ True   │ True   │ True   │ None    

Step 3: Download Images and Metadata
Batch Processing: Write a script to automatically download batches of images and their associated metadata.

Data Storage will utilize wounting a drive in google and creating files for the train, test, and validation images.

In [8]:
import requests
import os
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab import drive

In [9]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [32]:
# code block from kaggle documentation & chatgpt
# Specify the current path of kaggle.json file
current_path = '/content/kaggle.json'

# Desired path where the Kaggle API expects the kaggle.json file
desired_path = '/root/.kaggle/kaggle.json'

os.makedirs(os.path.dirname(desired_path), exist_ok=True)
os.rename(current_path, desired_path)

# Set the file's permissions to avoid a permissions error
os.chmod(desired_path, 600)


In [10]:
# For Train Set (Collection ID 60)
!mkdir -p "/content/myimages/training"
!isic image download --search "" --collections "60" --limit 0 "/content/myimages/training/"



Successfully downloaded 2,000 images to /content/myimages/training/.
Successfully wrote 2,000 metadata records to /content/myimages/training/metadata.csv.
Successfully wrote attributions to /content/myimages/training/attribution.txt.
Successfully wrote 1 license(s) to /content/myimages/training/licenses.


In [11]:
# For Test Set (Collection ID 69)
!mkdir -p "/content/myimages/test"
!isic image download --search "" --collections "69" --limit 0 "/content/myimages/test/"



Successfully downloaded 600 images to /content/myimages/test/.
Successfully wrote 600 metadata records to /content/myimages/test/metadata.csv.
Successfully wrote attributions to /content/myimages/test/attribution.txt.
Successfully wrote 1 license(s) to /content/myimages/test/licenses.


In [12]:
# For Validation Set (Collection ID 71)
!mkdir -p "/content/myimages/validation"
!isic image download --search "" --collections "71" --limit 0 "/content/myimages/validation/"


Successfully downloaded 150 images to /content/myimages/validation/.
Successfully wrote 150 metadata records to /content/myimages/validation/metadata.csv.
Successfully wrote attributions to /content/myimages/validation/attribution.txt.
Successfully wrote 1 license(s) to /content/myimages/validation/licenses.


Load the metadata

and

Explore the Metadata CSV: Load the metadata.csv files for training, test, and validation sets to understand the structure and types of data available. This step is crucial for preprocessing and feature selection.

In [13]:
# Paths to the metadata CSV files
train_metadata_path = '/content/myimages/training/metadata.csv'
test_metadata_path = '/content/myimages/test/metadata.csv'
validation_metadata_path = '/content/myimages/validation/metadata.csv'

# Load the metadata into pandas DataFrames
train_metadata = pd.read_csv(train_metadata_path)
test_metadata = pd.read_csv(test_metadata_path)
validation_metadata = pd.read_csv(validation_metadata_path)

# Display the first few rows of the training metadata to understand its structure
train_metadata.head()



,isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,diagnosis,...,family_hx_mm,image_type,lesion_id,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_0015295,Anonymous,CC-0,70.0,upper extremity,benign,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_2401252,False,NaN,IP_1751995,NaN,6688,4439,male
1,ISIC_0015284,Anonymous,CC-0,75.0,head/neck,malignant,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4598447,True,NaN,IP_0105331,NaN,6708,4459,male
2,ISIC_0015260,Anonymous,CC-0,55.0,upper extremity,benign,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_3146951,False,NaN,IP_5616465,NaN,6708,4439,female
3,ISIC_0015233,Anonymous,CC-0,60.0,head/neck,benign,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5410131,False,NaN,IP_4084040,NaN,6641,4401,male
4,ISIC_0015220,Anonymous,CC-0,75.0,head/neck,benign,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5213308,False,NaN,IP_4497472,NaN,6682,4441,male


In [14]:
# Load the metadata
train_metadata_path = '/content/myimages/training/metadata.csv'
train_metadata = pd.read_csv(train_metadata_path)

# Display the first few rows
print(train_metadata.head())

# Summary statistics for numerical columns
print(train_metadata.describe())

# Check for missing values
print(train_metadata.isnull().sum())

# Get a quick overview of the data types and non-null counts
print(train_metadata.info())


        isic_id attribution copyright_license  age_approx anatom_site_general  \
0  ISIC_0015295   Anonymous              CC-0        70.0     upper extremity   
1  ISIC_0015284   Anonymous              CC-0        75.0           head/neck   
2  ISIC_0015260   Anonymous              CC-0        55.0     upper extremity   
3  ISIC_0015233   Anonymous              CC-0        60.0           head/neck   
4  ISIC_0015220   Anonymous              CC-0        75.0           head/neck   

  benign_malignant  clin_size_long_diam_mm  concomitant_biopsy  \
0           benign                     NaN                True   
1        malignant                     NaN                True   
2           benign                     NaN                True   
3           benign                     NaN                True   
4           benign                     NaN                True   

        dermoscopic_type             diagnosis  ... family_hx_mm   image_type  \
0  contact non-polarized  seborrhei

In [26]:
# Dropping columns where more than 50% of the data is missing
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)
# Get a quick overview of the data types and non-null counts
train_metadata.head()

,isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,diagnosis,...,family_hx_mm,image_type,lesion_id,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_0015295,Anonymous,CC-0,70.0,upper extremity,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_2401252,False,NaN,IP_1751995,NaN,6688,4439,1
1,ISIC_0015284,Anonymous,CC-0,75.0,head/neck,NaN,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4598447,True,NaN,IP_0105331,NaN,6708,4459,1
2,ISIC_0015260,Anonymous,CC-0,55.0,upper extremity,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_3146951,False,NaN,IP_5616465,NaN,6708,4439,0
3,ISIC_0015233,Anonymous,CC-0,60.0,head/neck,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5410131,False,NaN,IP_4084040,NaN,6641,4401,1
4,ISIC_0015220,Anonymous,CC-0,75.0,head/neck,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5213308,False,NaN,IP_4497472,NaN,6682,4441,1


Preprocess the MetaData

In [27]:

# Load the metadata
train_metadata_path = '/content/myimages/training/metadata.csv'
train_metadata = pd.read_csv(train_metadata_path)

# Encode the target variable
label_encoder = LabelEncoder()
train_metadata['benign_malignant'] = label_encoder.fit_transform(train_metadata['benign_malignant'])

# Impute missing values for 'age_approx' with median
age_imputer = SimpleImputer(strategy='median')
train_metadata['age_approx'] = age_imputer.fit_transform(train_metadata[['age_approx']])

# Impute missing values for categorical data
cat_imputer = SimpleImputer(strategy='constant', fill_value='unknown')
categorical_columns = ['anatom_site_general', 'dermoscopic_type', 'sex']
train_metadata[categorical_columns] = cat_imputer.fit_transform(train_metadata[categorical_columns])

train_metadata.head(30)



,isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,diagnosis,...,family_hx_mm,image_type,lesion_id,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_0015295,Anonymous,CC-0,70.0,upper extremity,0,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_2401252,False,NaN,IP_1751995,NaN,6688,4439,male
1,ISIC_0015284,Anonymous,CC-0,75.0,head/neck,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4598447,True,NaN,IP_0105331,NaN,6708,4459,male
2,ISIC_0015260,Anonymous,CC-0,55.0,upper extremity,0,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_3146951,False,NaN,IP_5616465,NaN,6708,4439,female
3,ISIC_0015233,Anonymous,CC-0,60.0,head/neck,0,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5410131,False,NaN,IP_4084040,NaN,6641,4401,male
4,ISIC_0015220,Anonymous,CC-0,75.0,head/neck,0,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5213308,False,NaN,IP_4497472,NaN,6682,4441,male
5,ISIC_0015219,Anonymous,CC-0,50.0,upper extremity,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4271814,True,NaN,IP_4497472,NaN,6640,4441,male
6,ISIC_0015204,Anonymous,CC-0,30.0,lower extremity,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4810075,True,NaN,IP_9644248,NaN,6641,4420,male
7,ISIC_0015200,Anonymous,CC-0,50.0,upper extremity,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_8498118,True,NaN,IP_6972348,NaN,6661,4422,unknown
8,ISIC_0015190,Anonymous,CC-0,80.0,upper extremity,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_9653446,True,NaN,IP_4497472,NaN,6648,4419,male
9,ISIC_0015189,Anonymous,CC-0,35.0,posterior torso,3,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_3503592,True,NaN,IP_8634598,NaN,6668,4439,male


looks like we have an issue with out benign_malignant encoding, lets fix it

In [16]:
train_metadata['benign_malignant'] = train_metadata['benign_malignant'].map({'benign': 0, 'malignant': 1})


Lets adress some more of the missing data

In [23]:
# Correctly encode the target variable
train_metadata['benign_malignant'] = train_metadata['benign_malignant'].map({'benign': 0, 'malignant': 1})

# Impute missing values for 'age_approx' with median
age_imputer = SimpleImputer(strategy='median')
train_metadata['age_approx'] = age_imputer.fit_transform(train_metadata[['age_approx']])

# For categorical data with missing values, fill with 'unknown'
cat_imputer = SimpleImputer(strategy='constant', fill_value='unknown')
categorical_columns = ['anatom_site_general', 'dermoscopic_type', 'sex']
train_metadata[categorical_columns] = cat_imputer.fit_transform(train_metadata[categorical_columns])

# Example: Encoding 'sex' column (assuming it's categorical with no missing values after imputation)
label_encoder = LabelEncoder()
train_metadata['sex'] = label_encoder.fit_transform(train_metadata['sex'])

# Display the first few rows to verify changes
train_metadata.head()


,isic_id,attribution,copyright_license,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,concomitant_biopsy,dermoscopic_type,diagnosis,...,family_hx_mm,image_type,lesion_id,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_0015295,Anonymous,CC-0,70.0,upper extremity,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_2401252,False,NaN,IP_1751995,NaN,6688,4439,1
1,ISIC_0015284,Anonymous,CC-0,75.0,head/neck,NaN,NaN,True,contact non-polarized,melanoma,...,NaN,dermoscopic,IL_4598447,True,NaN,IP_0105331,NaN,6708,4459,1
2,ISIC_0015260,Anonymous,CC-0,55.0,upper extremity,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_3146951,False,NaN,IP_5616465,NaN,6708,4439,0
3,ISIC_0015233,Anonymous,CC-0,60.0,head/neck,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5410131,False,NaN,IP_4084040,NaN,6641,4401,1
4,ISIC_0015220,Anonymous,CC-0,75.0,head/neck,NaN,NaN,True,contact non-polarized,seborrheic keratosis,...,NaN,dermoscopic,IL_5213308,False,NaN,IP_4497472,NaN,6682,4441,1


Ok, lets start preprocessing the associated images



In [18]:
train_dir = "/content/myimages/training"
# Initialize list to hold file paths
file_paths = []

# Walk through the training directory to get file paths
for root, dirs, files in os.walk(train_dir):
    for file in files:
        if file.lower().endswith('.jpg') or file.endswith('.JPG'):  # Ensures all JPG files are included
            file_paths.append(os.path.join(root, file))

# Create a DataFrame
df = pd.DataFrame({'filepath': file_paths})



In [19]:
IMG_SIZE = (224, 224)

# Initialize an image data generator with some augmentation options
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=40,  # Random rotations
    width_shift_range=0.2,  # Random horizontal shifts
    height_shift_range=0.2,  # Random vertical shifts
    shear_range=0.2,  # Shearing
    zoom_range=0.2,  # Zooming
    horizontal_flip=True,  # Horizontal flipping
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

# Load images from the DataFrame
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    x_col='filepath',  # Column in DataFrame that contains the paths
    y_col=None,  # No labels
    target_size=IMG_SIZE,
    batch_size=20,
    class_mode=None  # No labels
)


Found 2000 validated image filenames.


2. Dataset Preparation

Image Preprocessing: Preprocess the images to a consistent size and format. normalize the image data (scaling pixel values to a range of 0 to 1).
Data Augmentation: To increase the size of dataset and improve the model's robustness, applying data augmentation techniques such as rotation, zoom, flip, and shift.
Labeling: Ensure each image is labeled with the correct type of skin growth. This might involve categorizing images by condition (e.g., benign, malignant, melanoma, etc.).

Define the model


In [20]:
# Load the MobileNetV2 model, excluding its top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))  # Adjusted image size
base_model.trainable = False  # Freeze the base model

# Add custom layers on top for our specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Adjust your ImageDataGenerator to resize images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Assuming you have adjusted your train_generator setup accordingly


9406464/9406464 [==============================] - 1s 0us/step


NameError: name 'Dense' is not defined

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# model = Sequential([
#     # Convolutional layers
#     Conv2D(32, (3, 3), activation='relu', input_shape=(1024, 1024, 3)),
#     MaxPooling2D(2, 2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     # Flattening the matrices into vectors
#     Flatten(),
#     # Dense layers
#     Dense(512, activation='relu'),
#     Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
# ])


compile the model

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class
              metrics=['accuracy'])


Train the model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // 20,  # Assuming a batch size of 20
    epochs=10
)


In [ ]:
print(len(train_data))

Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(test_data, verbose=2)
print(f"Test accuracy: {test_acc}")

Step 2: Merging Image and Metadata Features
To incorporate both image data and metadata in your model, consider a dual-input neural network architecture. One branch of the network will process the image data (using CNN layers), and the other branch will process the structured metadata (using dense layers). The outputs of these two branches can be concatenated and fed into additional layers to perform the final prediction.



Step 3: Model Architecture
Image Branch: Utilize a pre-trained CNN (like ResNet or Inception) as the base for feature extraction from images. Fine-tune this model with your specific dataset.
Metadata Branch: Design a simple feedforward neural network that takes in the processed metadata as input and outputs a representation of this data.
Combination and Prediction: Merge the outputs of both branches and feed them through one or more dense layers to produce a final prediction.

Step 4: Training
Loss Function: Use binary cross-entropy as the loss function since this is a binary classification task.
Metrics: Track accuracy and AUC (Area Under the Curve) to evaluate model performance.
Validation: Use a separate validation set (not seen by the model during training) to fine-tune hyperparameters and avoid overfitting.

Step 5: User Interface (UI)
A simple web-based UI (using Flask or Streamlit) allows users to upload lesion images, input relevant metadata, and receive a prediction. The UI should:

Enable image upload and metadata entry.
Preprocess the input data similarly to the training data.
Display the model's malignancy probability.

4. Integrate APIs

If using OpenAI's GPT-3, you'll interact with it through its API, sending requests with the classification results and receiving generated text.
If your application needs to store or retrieve data, consider integrating a database and accessing it through a REST API.


5. Build the Application Logic

Develop a user interface where users can upload photos of skin growths.
After uploading, the photo is sent to the image classification model to determine the type of growth.
The classification result is used to generate a prompt for the ChatGPT-like model, which then creates an informative response.
Display the response to the user along with any additional advice or recommendations.